In [ ]:
import tensorflow as tf

In [ ]:
tf.reset_default_graph()

MODEL_PATH = 'model.pb'
with tf.gfile.GFile(MODEL_PATH, 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())

graph = tf.Graph()
with graph.as_default():
    tf.import_graph_def(graph_def, name='import')

input_tensor = graph.get_tensor_by_name('import/images:0')
names = [
    'features', 'restored_images'
]
output_tensors = {n: graph.get_tensor_by_name('import/' + n + ':0') for n in names}

gpu_options = tf.GPUOptions(
    per_process_gpu_memory_fraction=1.0,
    visible_device_list='0'
)
config_proto = tf.ConfigProto(gpu_options=gpu_options, log_device_placement=False)
sess = tf.Session(graph=graph, config=config_proto)

def predict(image):
    # image - a RGB uint8 image
    
    start = time.perf_counter()
    feed_dict = {
        input_tensor: np.expand_dims(image, 0)
    }
    result = sess.run(output_tensors, feed_dict)
    inference_time = time.perf_counter() - start
    
    return result, inference_time

In [ ]:
from PIL import Image
import numpy as np
import time

In [ ]:
image = Image.open('/mnt/datasets/celeba_good_faces/val/183673.jpg')
image

In [ ]:
result, inference_time = predict(np.array(image))
inference_time

In [ ]:
r = (result['restored_images'][0]).astype('uint8')

In [ ]:
Image.fromarray(np.concatenate([np.array(image), r], axis=1))

In [ ]:
Image.fromarray(r)